In [1]:
import logging

logging.basicConfig(level=logging.INFO,
                    format='%(asctime)s  - %(levelname)s - %(message)s',
                    datefmt='%Y-%m-%d %H:%M:%S')

In [2]:
from optimex.converter import Converter
from optimex import optimizer
from optimex import postprocessing 

con_vs = "v2"
for optimex_type in ["static", "time_explicit_t50"]:

    converter = Converter(None)
    converter.unpickle_model_inputs(f"data/{optimex_type}_opt_inputs.pkl")

    converter.model_inputs.process_limits_max_default = 1e9
    if converter.model_inputs.process_limits_max is None:
        converter.model_inputs.process_limits_max = {}
    for year in range(2020, 2041):
        converter.model_inputs.process_limits_max.update({
            ("heat_prod_fuel_cell_cogen", year): 0.0,
        }) 
    for year in range(2020, 2051):
        converter.model_inputs.process_limits_max.update({
            ("heat_prod_bio_gas_cogen", year): 3.6e9,
            ("heat_prod_wood_chips_cogen", year): 3.6e9,
            ("heat_prod_municipal_waste", year): 3.6e9,
        })
    converter.model_inputs.cumulative_process_limits_max = {
        "heat_prod_municipal_waste": 1e10,
    }

    converter.pickle_model_inputs(f"data/{optimex_type}_opt_with_{con_vs}_con_inputs.pkl")
    model = optimizer.create_model(converter.model_inputs, name = f"optimex_{optimex_type}")
    model, results = optimizer.solve_model(model, compute_iis=True)

    results.write(filename=f"data/{optimex_type}_{con_vs}_results.csv")
    pp = postprocessing.PostProcessor(model)
    df_scaling = pp.get_scaling()
    df_scaling.to_csv(f"data/{optimex_type}_{con_vs}_opt_scaling.csv")
    fig_scale = pp.plot_scaling(f"figures/{optimex_type}_{con_vs}_opt_scaling.png")
    dfs_production = pp.get_production()
    fig_prod = pp.plot_production(f"figures/{optimex_type}_{con_vs}_opt_production.png")



2025-01-23 02:30:13  - INFO - Creating sets
2025-01-23 02:30:13  - INFO - Creating parameters
2025-01-23 02:30:13  - INFO - Creating variables
2025-01-23 02:30:13  - INFO - Creating expressions
2025-01-23 02:36:45  - INFO - Creating constraints
2025-01-23 02:36:45  - INFO - Creating objective function


Set parameter Username
Academic license - for non-commercial use only - expires 2026-01-20
Read LP format model from file C:\Users\LAYTON~1\AppData\Local\Temp\tmpqhxtquz5.pyomo.lp
Reading time = 0.02 seconds
x1: 1564 rows, 651 columns, 14062 nonzeros
Set parameter MIPGap to value 0.01
Gurobi Optimizer version 9.5.2 build v9.5.2rc0 (win64)
Thread count: 6 physical cores, 6 logical processors, using up to 6 threads
Optimize a model with 1564 rows, 651 columns and 14062 nonzeros
Model fingerprint: 0x577dfd5b
Coefficient statistics:
  Matrix range     [2e-02, 1e+01]
  Objective range  [8e+00, 5e+09]
  Bounds range     [1e+00, 1e+00]
  RHS range        [6e+07, 1e+10]
         Consider reformulating model or setting NumericFocus parameter
         to avoid numerical issues.
Presolve removed 1513 rows and 396 columns
Presolve time: 0.00s
Presolved: 51 rows, 255 columns, 4329 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    0.0000000e+00   2.160627e+10   0.